In [1]:
from __future__ import print_function
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from collections import Counter, OrderedDict

from PIL import Image as PImage
from os import listdir
from pickle import dump
import matplotlib.pyplot as plt
import PIL, cv2, os, json, glob, h5py, keras, csv, gc, random
from IPython.display import SVG

import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard
from keras.applications.vgg16 import preprocess_input
from keras.utils.vis_utils import plot_model, model_to_dot

#import seaborn as sns

Using TensorFlow backend.


In [2]:
train_path = '../data/aia-picture-classification1/train'
test_path = '../data/aia-picture-classification1/test'
model_path = '../data/aia-picture-classification1/model'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5729061521679370824
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15787999232
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15648314789286185253
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 15870492672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16349580412776724725
physical_device_desc: "device: 1, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:05.0, compute capability: 6.0"
]


In [4]:
label_list = !ls {train_path}
label_list

['bedroom',
 'CALsuburb',
 'coast',
 'forest',
 'highway',
 'industrial',
 'insidecity',
 'kitchen',
 'livingroom',
 'mountain',
 'opencountry',
 'PARoffice',
 'store',
 'street',
 'tallbuilding']

In [5]:
mtype_dict = {}
mtype_list = [None] * len(label_list)
with open('../data/aia-picture-classification1/target_to_number.txt', newline='') as csvfile:
    _dict = csv.DictReader(csvfile)
    for idx, row in enumerate(_dict):
        mtype_dict[row['type'].strip()] = int(row[' index'].strip())
        mtype_list[int(row[' index'].strip())] = row['type'].strip()

In [6]:
mtype_list

['kitchen',
 'street',
 'industrial',
 'insidecity',
 'forest',
 'livingroom',
 'opencountry',
 'PARoffice',
 'mountain',
 'CALsuburb',
 'coast',
 'store',
 'bedroom',
 'tallbuilding',
 'highway']

In [7]:
from keras.layers import *
from keras.optimizers import *
from keras.applications import *

from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Dropout, BatchNormalization
from keras import regularizers
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

import keras.applications.xception as Xception
import keras.applications.vgg16 as VGG16
import keras.applications.resnet50 as resnet50
import keras.applications.inception_v3 as InceptionV3
import keras.applications.densenet as densenet

#import InceptionResNetV2, preprocess_input, decode_predictions resnet50

In [8]:
model_param = \
{
    'Xception': {'target_size': (299, 299),
                  'preprocess_input':Xception.preprocess_input,
                  'model_obj':Xception.Xception},
    'VGG16': {'target_size': (224, 224),
                 'preprocess_input':VGG16.preprocess_input,
                 'model_obj':VGG16.VGG16},
    'ResNet50': {'target_size': (224, 224),
                    'preprocess_input':resnet50.preprocess_input,
                    'model_obj':resnet50.ResNet50},
    'InceptionV3': {'target_size': (299, 299),
                    'preprocess_input':InceptionV3.preprocess_input,
                    'model_obj':InceptionV3.InceptionV3},
    'DenseNet': {'target_size': (224, 224),
              'preprocess_input':densenet.preprocess_input,
              'model_obj':densenet.DenseNet201}
}

In [9]:
def set_callback(model_name):
    top_weights_path = os.path.join(model_path, 'top_model_weights_{}.h5'.format(model_name))
    csv_path = os.path.join(model_path, 'top_model_csv_{}.h5'.format(model_name))
    callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='loss', patience=15, verbose=0),
        CSVLogger(csv_path, separator=',', append=False)
    ]
    return (top_weights_path,csv_path,callbacks_list)
#tensor_board = callbacks.TensorBoard()
#set_callback('resnet_v2')

In [10]:
nb_classes = len(mtype_dict)
batch_size = 16
nb_epoch = 60

In [17]:
class Model_Generator():
    model_param = \
    {
        'Xception': {'target_size': (299, 299),
                      'preprocess_input':Xception.preprocess_input,
                      'model_obj':Xception.Xception,
                      'optimizer':'nadam'
                    },
        'VGG16': {'target_size': (224, 224),
                     'preprocess_input':VGG16.preprocess_input,
                     'model_obj':VGG16.VGG16,
                     'rescale':1./255,
                     'optimizer':'nadam'
                 },
        'ResNet50': {'target_size': (224, 224),
                        'preprocess_input':resnet50.preprocess_input,
                        'model_obj':resnet50.ResNet50,
                        'optimizer':'nadam'
                    },
        'InceptionV3': {'target_size': (299, 299),
                        'preprocess_input':InceptionV3.preprocess_input,
                        'model_obj':InceptionV3.InceptionV3,
                        'optimizer':'nadam'
                       },
        'DenseNet': {'target_size': (224, 224),
                     'preprocess_input':densenet.preprocess_input,
                     'model_obj':densenet.DenseNet201,
                     'optimizer':'nadam'
                    }
    }
    def __init__(self,name):
        self.param = self.model_param[name] 
        self.modelname = name
        self.preprocess_input = self.param['preprocess_input']
        (self.img_width, self.img_height) = self.param['target_size']
        self.model = self.param['model_obj'](input_shape=(self.img_width, self.img_height, 3), weights='imagenet', include_top=False)
        for layer in self.model.layers:
            layer.trainable = False
            
    def get_model(self):
        if self.modelname == 'Xception':
            x = self.model.output
            x = GlobalMaxPooling2D()(x)
            x = BatchNormalization()(x)
            x = Dropout(0.2)(x)
            output = Dense(nb_classes, activation='softmax', name='softmax')(x)
            modelf = Model(self.model.input, output)
            modelf.compile(optimizer='nadam',
                          loss=categorical_crossentropy, metrics=['accuracy',])
        if self.modelname == 'VGG16':
            x = self.model.output
            x = Flatten()(x)
            x = Dropout(0.2)(x)
            x = Dense(256, activation='relu')(x)
            x = Dropout(0.2)(x)
            output = Dense(nb_classes, activation='softmax', name='softmax')(x)
            modelf = Model(self.model.input, output)
            modelf.compile(optimizer='nadam',
                          loss=categorical_crossentropy, metrics=['accuracy',])
            
        if self.modelname == 'ResNet50':
            x = self.model.output
            x = GlobalAveragePooling2D()(x)
            x = BatchNormalization()(x)
            x = Dropout(0.2)(x)
            output = Dense(nb_classes, activation='softmax', name='softmax')(x)
            modelf = Model(self.model.input, output)
            modelf.compile(optimizer='nadam',
                          loss=categorical_crossentropy, metrics=['accuracy',])
            
        if self.modelname == 'InceptionV3':
            x = self.model.output
            x = GlobalMaxPooling2D()(x)
            x = BatchNormalization()(x)
            x = Dropout(0.2)(x)
            output = Dense(nb_classes, activation='softmax', name='softmax')(x)
            modelf = Model(self.model.input, output)
            modelf.compile(optimizer='nadam',
                          loss=categorical_crossentropy, metrics=['accuracy',])            
                
        if self.modelname == 'DenseNet':
            x = self.model.output
            x = GlobalAveragePooling2D()(x)
            x = BatchNormalization()(x)
            x = Dropout(0.2)(x)
            output = Dense(nb_classes, activation='softmax', name='softmax')(x)
            modelf = Model(self.model.input, output)
            modelf.compile(optimizer='nadam',
                          loss=categorical_crossentropy, metrics=['accuracy',])                
                
        return modelf

In [20]:
for model_name in model_param:
    this_model = Model_Generator(model_name)
    seed = random.randint(1, 2000)
    print("Start to train %s" % (model_name))
    preprocess_input = this_model.param['preprocess_input']
    (img_width, img_height) = this_model.param['target_size']
    model = this_model.get_model()
    rescale = None
    if 'rescale' in this_model.param:
        rescale = this_model.param['rescale']

    data_gen = image.ImageDataGenerator(validation_split=0.1,
                                        fill_mode="nearest",
                                        rotation_range=20,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=[0.8, 1.4],
                                        horizontal_flip=True,
                                        rescale=rescale,
                                        preprocessing_function=preprocess_input)

    train_generator = data_gen.flow_from_directory(train_path,
                                        target_size=(img_width, img_height),
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=True, seed=seed, subset="training")

    validation_generator = data_gen.flow_from_directory(train_path,
                                        target_size=(img_width, img_height),
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=True, seed=seed, subset="validation")

    (top_weights_path,csv_path,callbacks_list) = set_callback(model_name)

    print("Start to fit %s" % (model_name))
    history = model.fit_generator(train_generator,
                        epochs=2,
                        validation_data=validation_generator,
                        callbacks=callbacks_list,
                        workers=8,
                        use_multiprocessing=True)    
    
    
    print("End to fit %s" % (model_name))
    del history
    del this_model, model, train_generator, validation_generator, data_gen
    gc.collect()

Start to train Xception
{'target_size': (299, 299), 'preprocess_input': <function preprocess_input at 0x7f0761021268>, 'model_obj': <function Xception at 0x7f07610211e0>, 'optimizer': 'nadam'}
Found 2692 images belonging to 15 classes.
Found 293 images belonging to 15 classes.
Start to fit Xception
Epoch 1/2
169/169 [==============================] - 60s 352ms/step - loss: 1.3184 - acc: 0.5991 - val_loss: 1.2802 - val_acc: 0.6724

Epoch 00001: val_acc improved from -inf to 0.67235, saving model to ../data/aia-picture-classification1/model/top_model_weights_Xception.h5
Epoch 2/2
169/169 [==============================] - 56s 331ms/step - loss: 0.9493 - acc: 0.7204 - val_loss: 1.3550 - val_acc: 0.6621

Epoch 00002: val_acc did not improve from 0.67235
End to fit Xception
Start to train VGG16
{'target_size': (224, 224), 'preprocess_input': <function preprocess_input at 0x7f0761019268>, 'model_obj': <function VGG16 at 0x7f076100ae18>, 'rescale': 0.00392156862745098, 'optimizer': 'nadam'}
0

In [48]:
for model_name in model_param:
    weights_path = os.path.join(model_path, 'top_model_weights_{}.h5'.format(model_name))
    this_model = Model_Generator(model_name)
    preprocess_input = this_model.param['preprocess_input']
    (img_width, img_height) = this_model.param['target_size']
    model = this_model.get_model()
    rescale = None
    if 'rescale' in this_model.param:
        rescale = this_model.param['rescale']
    
    test_datagen = image.ImageDataGenerator(rescale=rescale, fill_mode="nearest",
                                       preprocessing_function=preprocess_input)

    test_generator =  test_datagen.flow_from_directory(train_path, 
                                         target_size=(img_width, img_height), 
                                         batch_size=batch_size,
                                         shuffle=False,
                                         class_mode=None)
    
    
    model.load_weights(weights_path)
    X_l_vec = model.predict_generator(generator=test_generator, workers=8)
    model_param[model_name]['predict'] = X_l_vec

Found 2985 images belonging to 15 classes.
Found 2985 images belonging to 15 classes.
Found 2985 images belonging to 15 classes.
Found 2985 images belonging to 15 classes.
Found 2985 images belonging to 15 classes.


In [49]:
X_l_vec = np.hstack((model_param['DenseNet']['predict'],
                     model_param['InceptionV3']['predict'],
                     model_param['Xception']['predict'],
                     model_param['ResNet50']['predict'],
                     model_param['VGG16']['predict']))

In [52]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [53]:
X = X_l_vec
Y = test_generator.classes

In [54]:
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [63]:
model_g = XGBClassifier()
model_g.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [64]:
# make predictions for test data
y_pred = model_g.predict(X_test)
predictions = [round(value) for value in y_pred]

/home/wayne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [65]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.97%


In [58]:
for model_name in model_param:
    weights_path = os.path.join(model_path, 'top_model_weights_{}.h5'.format(model_name))
    this_model = Model_Generator(model_name)
    preprocess_input = this_model.param['preprocess_input']
    (img_width, img_height) = this_model.param['target_size']
    model = this_model.get_model()
    rescale = None
    if 'rescale' in this_model.param:
        rescale = this_model.param['rescale']
    
    test_datagen = image.ImageDataGenerator(rescale=rescale, fill_mode="nearest",
                                       preprocessing_function=preprocess_input)

    test_generator =  test_datagen.flow_from_directory(test_path, 
                                         target_size=(img_width, img_height), 
                                         batch_size=batch_size,
                                         shuffle=False,
                                         class_mode=None)
    
    
    model.load_weights(weights_path)
    X_t_vec = model.predict_generator(generator=test_generator, workers=8)
    model_param[model_name]['predict_test'] = X_t_vec

Found 1500 images belonging to 1 classes.
Found 1500 images belonging to 1 classes.
Found 1500 images belonging to 1 classes.
Found 1500 images belonging to 1 classes.
Found 1500 images belonging to 1 classes.


In [59]:
X_t_vec = np.hstack((model_param['DenseNet']['predict_test'],
                     model_param['InceptionV3']['predict_test'],
                     model_param['Xception']['predict_test'],
                     model_param['ResNet50']['predict_test'],
                     model_param['VGG16']['predict_test']))

In [60]:
X_t_vec.shape

(1500, 75)

In [66]:
y_pred_test = model_g.predict(X_t_vec)
predictions = [round(value) for value in y_pred_test]

/home/wayne/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [68]:
predictions[0:10]

[1, 4, 11, 10, 3, 7, 4, 8, 10, 7]

In [69]:
fnames = [_f.split('/')[1].replace('.jpg','') for _f in test_generator.filenames]

In [70]:
fnames[0]

'001d4c8d70ebf7f025fccf256324d3d5ad3560faee1cdf8c7115f5eb033bc3d2'

In [73]:
data_gen = image.ImageDataGenerator(validation_split=0.1,
                                        fill_mode="nearest",
                                        rotation_range=20,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=[0.8, 1.4],
                                        horizontal_flip=True,
                                        rescale=rescale,
                                        preprocessing_function=preprocess_input)

train_generator = data_gen.flow_from_directory(train_path,
                                    target_size=(img_width, img_height),
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True, seed=seed, subset="training")

Found 2692 images belonging to 15 classes.


In [74]:
class_list = [None] * len(train_generator.class_indices)
for _item in train_generator.class_indices:
    class_list[train_generator.class_indices[_item]] = _item

In [75]:
train_generator.class_indices

{'CALsuburb': 0,
 'PARoffice': 1,
 'bedroom': 2,
 'coast': 3,
 'forest': 4,
 'highway': 5,
 'industrial': 6,
 'insidecity': 7,
 'kitchen': 8,
 'livingroom': 9,
 'mountain': 10,
 'opencountry': 11,
 'store': 12,
 'street': 13,
 'tallbuilding': 14}

In [77]:
mtype_dict = {}
mtype_list = [None] * len(label_list)
with open('../data/aia-picture-classification1/target_to_number.txt', newline='') as csvfile:
    _dict = csv.DictReader(csvfile)
    for idx, row in enumerate(_dict):
        mtype_dict[row['type'].strip()] = int(row[' index'].strip())
        mtype_list[int(row[' index'].strip())] = row['type'].strip()

In [80]:
cateidxs = [mtype_dict[class_list[_p]] for _p in predictions]

In [81]:
percentile_list = pd.DataFrame(
    {'id': fnames,
     'class': cateidxs
    })

In [82]:
percentile_list.to_csv('out_stacking_1.csv',encoding='utf-8', index=False,columns=["id","class"])